## What it does

"This code simulates a Tic-Tac-Toe game where two AI agents compete using different strategies like Minimax, Alpha-Beta Pruning, Expectimax, and Q-learning. The goal is to demonstrate how AI strategies make decisions and play the game optimally."

## Strategies used 

Minimax: 
A basic algorithm that recursively explores all possible moves to maximize the current player's score and minimize the opponent's score. It returns the best move based on this exploration.

Alpha-Beta Pruning:
An optimized version of the Minimax algorithm that prunes branches of the search tree, reducing the number of nodes evaluated and speeding up the decision-making process.

Expectimax: 
Similar to Minimax but accounts for probabilistic outcomes, averaging the possible results when it's the opponent's turn.

Q-Learning:
A reinforcement learning algorithm where an agent learns the optimal strategy by updating a Q-table based on experiences (state, action, reward, next state). The agent can explore new moves or exploit known moves based on an exploration rate.

## Tic Tac Toe Game

In [18]:
import time
import numpy as np

# TicTacToeGame class handles the game mechanics and rules
class TicTacToeGame:
    def __init__(self):
        # Initialize the game board with empty spaces
        self.board = [' '] * 9

    def display_board(self):
        # Display the current state of the board
        for i in range(0, 9, 3):
            print(f"| {self.board[i]} | {self.board[i+1]} | {self.board[i+2]} |")
            if i < 6:
                print("-------------")

    def is_winner(self, marker):
        # Check if the given marker (X or O) has a winning combination
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]             # diagonals
        ]
        return any(all(self.board[pos] == marker for pos in combo) for combo in winning_combinations)

    def is_draw(self):
        # Check if the game has ended in a draw (no empty spaces left)
        return all(spot != ' ' for spot in self.board)

    def valid_moves(self):
        # Return a list of all valid positions where a move can be made
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def make_move(self, position, marker):
        # Place the marker (X or O) at the specified position if valid
        if self.board[position] == ' ':
            self.board[position] = marker
            return True
        return False


# Agent class represents an AI player with a specific strategy
class Agent:
    def __init__(self, name, strategy):
        # Initialize the agent with a name and a chosen strategy
        self.name = name
        self.strategy = strategy

    def choose_move(self, game, marker):
        # Determine the agent's next move based on its strategy
        if self.strategy == "minimax":
            return self.minimax_move(game, marker)
        elif self.strategy == "alphabeta":
            return self.alphabeta_move(game, marker)
        elif self.strategy == "expectimax":
            return self.expectimax_move(game, marker)
        elif self.strategy == "qlearning":
            return self.q_learning_move(game, marker)
        else:
            raise ValueError(f"Unknown strategy: {self.strategy}")

    def minimax_move(self, game, marker):
        # Use the Minimax algorithm to calculate the optimal move
        opponent_marker = 'O' if marker == 'X' else 'X'
        best_score = -float('inf')
        best_move = None

        for move in game.valid_moves():
            # Simulate the move
            game.make_move(move, marker)
            # Evaluate the move using the Minimax algorithm
            score = self.minimax(game, opponent_marker, False)
            # Undo the move
            game.make_move(move, ' ')
            # Update the best move if the score is higher
            if score > best_score:
                best_score = score
                best_move = move

        return best_move

    def minimax(self, game, marker, is_maximizing):
        # Recursive Minimax function to evaluate the game state
        opponent_marker = 'O' if marker == 'X' else 'X'

        # Base cases: check for win, loss, or draw
        if game.is_winner(marker):
            return 1
        if game.is_winner(opponent_marker):
            return -1
        if game.is_draw():
            return 0

        # Recursive case: maximize or minimize the score
        if is_maximizing:
            best_score = -float('inf')
            for move in game.valid_moves():
                game.make_move(move, marker)
                score = self.minimax(game, opponent_marker, False)
                game.make_move(move, ' ')
                best_score = max(best_score, score)
            return best_score
        else:
            best_score = float('inf')
            for move in game.valid_moves():
                game.make_move(move, marker)
                score = self.minimax(game, opponent_marker, True)
                game.make_move(move, ' ')
                best_score = min(best_score, score)
            return best_score

    def alphabeta_move(self, game, marker):
        # Use Alpha-Beta Pruning to improve the Minimax algorithm
        opponent_marker = 'O' if marker == 'X' else 'X'
        best_score = -float('inf')
        best_move = None

        for move in game.valid_moves():
            # Simulate the move
            game.make_move(move, marker)
            # Evaluate the move using the Alpha-Beta algorithm
            score = self.alphabeta(game, opponent_marker, -float('inf'), float('inf'), False)
            # Undo the move
            game.make_move(move, ' ')
            # Update the best move if the score is higher
            if score > best_score:
                best_score = score
                best_move = move

        return best_move

    def alphabeta(self, game, marker, alpha, beta, is_maximizing):
        # Recursive Alpha-Beta Pruning function to evaluate the game state
        opponent_marker = 'O' if marker == 'X' else 'X'

        # Base cases: check for win, loss, or draw
        if game.is_winner(marker):
            return 1
        if game.is_winner(opponent_marker):
            return -1
        if game.is_draw():
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for move in game.valid_moves():
                game.make_move(move, marker)
                score = self.alphabeta(game, opponent_marker, alpha, beta, False)
                game.make_move(move, ' ')
                best_score = max(best_score, score)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break  # Beta cutoff
            return best_score
        else:
            best_score = float('inf')
            for move in game.valid_moves():
                game.make_move(move, marker)
                score = self.alphabeta(game, opponent_marker, alpha, beta, True)
                game.make_move(move, ' ')
                best_score = min(best_score, score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break  # Alpha cutoff
            return best_score

    def expectimax_move(self, game, marker):
        # Use the Expectimax algorithm to calculate the optimal move
        opponent_marker = 'O' if marker == 'X' else 'X'
        best_score = -float('inf')
        best_move = None

        for move in game.valid_moves():
            game.make_move(move, marker)
            score = self.expectimax(game, opponent_marker, False)
            game.make_move(move, ' ')
            if score > best_score:
                best_score = score
                best_move = move

        return best_move

    def expectimax(self, game, marker, is_maximizing):
        # Recursive Expectimax function to evaluate the game state
        opponent_marker = 'O' if marker == 'X' else 'X'

        if game.is_winner(marker):
            return 1
        if game.is_winner(opponent_marker):
            return -1
        if game.is_draw():
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for move in game.valid_moves():
                game.make_move(move, marker)
                score = self.expectimax(game, opponent_marker, False)
                game.make_move(move, ' ')
                best_score = max(best_score, score)
            return best_score
        else:
            total_score = 0
            valid_moves = game.valid_moves()
            for move in valid_moves:
                game.make_move(move, opponent_marker)
                score = self.expectimax(game, marker, True)
                game.make_move(move, ' ')
                total_score += score
            return total_score / len(valid_moves)

    def q_learning_move(self, game, marker, learning_rate=0.1, discount_factor=0.9, exploration_rate=1.0):
        # Q-learning move selection and learning
        state = tuple(game.board)  # The current state represented as a tuple of board positions
        if random.uniform(0, 1) < exploration_rate:
            # Explore: choose a random move
            return random.choice(game.valid_moves())
        else:
            # Exploit: choose the best move based on Q-table
            q_values = {action: self.q_table.get((state, action), 0) for action in game.valid_moves()}
            best_action = max(q_values, key=q_values.get)
            return best_action

    def update_q_table(self, state, action, reward, next_state, learning_rate=0.1, discount_factor=0.9):
        # Update the Q-value using the Q-learning formula
        old_q_value = self.q_table.get((state, action), 0)
        future_q_value = max(self.q_table.get((next_state, a), 0) for a in game.valid_moves())
        new_q_value = old_q_value + learning_rate * (reward + discount_factor * future_q_value - old_q_value)
        self.q_table[(state, action)] = new_q_value


# Function to simulate a single game between two agents
def play_game(agent1, agent2):
    game = TicTacToeGame()  # Create a new game instance
    current_agent, current_marker = agent1, 'X'
    other_agent, other_marker = agent2, 'O'

    while True:
        # Display the current board state
        game.display_board()
        # Get the current agent's move
        move = current_agent.choose_move(game, current_marker)
        game.make_move(move, current_marker)

        # Check for a win or draw
        if game.is_winner(current_marker):
            game.display_board()
            print(f"Winner: {current_agent.name}!")
            return current_agent.name
        if game.is_draw():
            game.display_board()
            print("It's a draw!")
            return "Draw"

        # Switch turns
        current_agent, other_agent = other_agent, current_agent
        current_marker, other_marker = other_marker, current_marker


# Main program to simulate a tournament
if __name__ == "__main__":
    print("Welcome to the Tic-Tac-Toe AI Tournament!")
    print("Available strategies: Minimax, AlphaBeta, Expectimax, QLearning")

    # Allow the user to choose agents and strategies
    agent1_name = input("Enter name for Agent 1: ")
    agent1_strategy = input("Enter strategy for Agent 1 (minimax, alphabeta, expectimax, qlearning): ").lower()

    agent2_name = input("Enter name for Agent 2: ")
    agent2_strategy = input("Enter strategy for Agent 2 (minimax, alphabeta, expectimax, qlearning): ").lower()

    agent1 = Agent(agent1_name, agent1_strategy)
    agent2 = Agent(agent2_name, agent2_strategy)

    # Run a tournament with multiple games
    results = {"Agent 1 Wins": 0, "Agent 2 Wins": 0, "Draws": 0}
    num_games = int(input("Enter the number of games to simulate: "))

    start_time = time.time()
    for _ in range(num_games):
        result = play_game(agent1, agent2)
        if result == agent1.name:
            results["Agent 1 Wins"] += 1
        elif result == agent2.name:
            results["Agent 2 Wins"] += 1
        else:
            results["Draws"] += 1

    end_time = time.time()

    # Display tournament results
    print("\nTournament Results:")
    print(results)
    print(f"Average game duration: {(end_time - start_time) / num_games:.2f} seconds")



Welcome to the Tic-Tac-Toe AI Tournament!
Available strategies: Minimax, AlphaBeta, Expectimax, QLearning


Enter name for Agent 1:  e
Enter strategy for Agent 1 (minimax, alphabeta, expectimax, qlearning):  qlearning
Enter name for Agent 2:  r
Enter strategy for Agent 2 (minimax, alphabeta, expectimax, qlearning):  expectimax
Enter the number of games to simulate:  12


|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
| X |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
| X | O | O |
-------------
| X | X | O |
-------------
| O | X | X |
It's a draw!
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
|   |   |   |
-------------
|   |   | X |
-------------
|   |   |   |
| O | O | X |
-------------
| X | O | X |
-------------
| O | X | X |
It's a draw!
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
|   |   |   |
-------------
|   |   |   |
-------------
|   |   | X |
| O | O | X |
-------------
| X | O | O |
-------------
| X | X | X |
It's a draw!
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
|   |   |   |
-------------
|   |   |   |
-------------
|   |   | X |
| O | O | X |
-------------
| X | O | O |
-------------
| X | X | X |
It's a draw!
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
|   |   |   |
-------------
| X |   | 